In [1]:
import tensorflow as tf
import numpy as np
import sys

In [2]:
import EER_mod
import read_mod

In [3]:
# Provide the file with the information about the model to be used
fl_model = open('model_S10_speeches30.dat','r')

for l in fl_model.readlines():
    print(l.strip())
    
fl_model.seek(0)

model_name =  model_S10_speeches30
thres =  0.9737797975540161
speech =  speeches/19-198-0000.npy
speech =  speeches/19-198-0001.npy
speech =  speeches/19-198-0002.npy
speech =  speeches/26-495-0000.npy
speech =  speeches/26-495-0001.npy
speech =  speeches/26-495-0002.npy
speech =  speeches/27-123349-0000.npy
speech =  speeches/27-123349-0001.npy
speech =  speeches/27-123349-0002.npy
speech =  speeches/32-21625-0000.npy
speech =  speeches/32-21625-0001.npy
speech =  speeches/32-21625-0002.npy
speech =  speeches/39-121914-0000.npy
speech =  speeches/39-121914-0001.npy
speech =  speeches/39-121914-0002.npy
speech =  speeches/40-121026-0000.npy
speech =  speeches/40-121026-0001.npy
speech =  speeches/40-121026-0002.npy
speech =  speeches/60-121082-0000.npy
speech =  speeches/60-121082-0001.npy
speech =  speeches/60-121082-0002.npy
speech =  speeches/78-368-0000.npy
speech =  speeches/78-368-0001.npy
speech =  speeches/78-368-0002.npy
speech =  speeches/83-11691-0000.npy
speech =  speeches

0

In [4]:
# Provide the file with the speeches of the speakers to be compared
fl_speeches = open('file_speeches.txt','r')

for l in fl_speeches.readlines():
    print(l.strip())
    
fl_speeches.seek(0)

speeches/19-198-0002.npy
speeches/19-198-0003.npy
speeches/19-198-0004.npy
speeches/26-495-0003.npy
speeches/26-495-0005.npy
speeches/27-123349-0003.npy
speeches/27-123349-0006.npy
speeches/89-218-0001.npy
speeches/89-218-0002.npy
speeches/103-1240-0001.npy
speeches/103-1240-0002.npy
speeches/118-121721-0001.npy
speeches/118-121721-0002.npy


0

In [5]:
# Load the model, threshold, speeches, labels, and some usefull information
model_name, thres, speeches, indx_i_f, labels = read_mod.read_model(fl_model, fl_speeches)

# Known speakers and speeches are highlighted with green

Known speakers are ['19', '26', '27', '32', '39', '40', '60', '78', '83', '87']
0019-000198-0002
0019-000198-0003
0019-000198-0004
0026-000495-0003
0026-000495-0005
0027-123349-0003
0027-123349-0006
0089-000218-0001
0089-000218-0002
0103-001240-0001
0103-001240-0002
0118-121721-0001
0118-121721-0002


In [8]:
# load model
model = tf.keras.models.load_model(model_name)

# Concatenate speeches and create feature vectors for them
data = np.concatenate(speeches)

# Get predictions
features = model.predict(data, verbose=0)

# Average features
# Alternatively, one can try to compare the averaged outputs from openl3
n_sp = len( speeches )

av_emb = []
for i in range(n_sp):
    av_emb.append( np.average(features[indx_i_f[i][0]:indx_i_f[i][1]+1,:],axis=0) )

In [10]:
# Compare speakers
print('\nThreshold = ', f'{thres: .3f}')
for i in range(n_sp):
    for j in range(i+1, n_sp):
        cos_av = EER_mod.cos_dist(av_emb[i], av_emb[j])
        
        answer = cos_av > thres
        correct_answer = labels[i].split('-')[0] == labels[j].split('-')[0]

        if answer == correct_answer:
            if answer:
                print(labels[i], '  ', labels[j], f'{cos_av: .3f}', 'same')
            else:
                print(labels[i], '  ', labels[j], f'{cos_av: .3f}', 'diff')
        else:
            if answer:
                print(labels[i], '  ', labels[j], f'{cos_av: .3f}', 'same', 
                      '\033[97;41m' + 'WRONG!!!' + '\033[0m')
            else:
                print(labels[i], '  ', labels[j], f'{cos_av: .3f}', 'diff', 
                      '\033[97;41m' + 'WRONG!!!' + '\033[0m')


Threshold =   0.974
0019-000198-0002    0019-000198-0003  0.997 same
0019-000198-0002    0019-000198-0004  0.977 same
0019-000198-0002    0026-000495-0003  0.267 diff
0019-000198-0002    0026-000495-0005  0.351 diff
0019-000198-0002    0027-123349-0003  0.252 diff
0019-000198-0002    0027-123349-0006  0.237 diff
0019-000198-0002    0089-000218-0001  0.907 diff
0019-000198-0002    0089-000218-0002  0.939 diff
0019-000198-0002    0103-001240-0001  0.807 diff
0019-000198-0002    0103-001240-0002  0.841 diff
0019-000198-0002    0118-121721-0001  0.760 diff
0019-000198-0002    0118-121721-0002  0.741 diff
0019-000198-0003    0019-000198-0004  0.989 same
0019-000198-0003    0026-000495-0003  0.288 diff
0019-000198-0003    0026-000495-0005  0.373 diff
0019-000198-0003    0027-123349-0003  0.297 diff
0019-000198-0003    0027-123349-0006  0.281 diff
0019-000198-0003    0089-000218-0001  0.915 diff
0019-000198-0003    0089-000218-0002  0.947 diff
0019-000198-0003    0103-001240-0001  0.837 diff

All speeches were identified correctly.